In [1]:
import pandas as pd
import torch
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, IntervalStrategy


c:\Users\admin\PycharmProjects\test_task\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Чтение данных

In [3]:
df = pd.read_csv('meatinfo.csv', delimiter=';')
df

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина
...,...,...
17888,"Ягнятина, шея, бк",Ягнятина
17889,Язык ягненка (н.зеландия) Отварные языки ягнят...,Ягнятина
17890,"Ягнятина, язык, зачищ. цена 100 руб.",Ягнятина
17891,"Як, задние части, 1 категория цена 550 руб.",Як


In [4]:
value_counts = df['mtype'].value_counts()
frequent_values = value_counts[value_counts > 500].index
df = df.loc[df['mtype'].isin(frequent_values)]
df

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина
...,...,...
17816,"Цыпленок, четвертина, задняя",Цыпленок
17817,"Цыпленок, четвертина, задняя цена 100 руб.",Цыпленок
17818,ЦБ Шеи п/ф Свеженка ГОСТ зам пак Шеи куриные П...,Цыпленок
17819,"Цыпленок, шея, без кожи",Цыпленок


In [5]:
train_df, test_df = train_test_split(df, train_size=0.8)
train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [6]:
def create_label_encoder(values):
    labels_to_code = {}
    for i, label in enumerate(sorted(values)):
        labels_to_code[label] = i
    code_to_labels = {code: label for label, code in labels_to_code.items()}
    return labels_to_code, code_to_labels

In [7]:
label_to_code, code_to_label = create_label_encoder(pd.unique(train_df['mtype']))
print(label_to_code)
print(code_to_label)

{'Баранина': 0, 'Говядина': 1, 'Индейка': 2, 'Кура': 3, 'Свинина': 4, 'Цыпленок': 5}
{0: 'Баранина', 1: 'Говядина', 2: 'Индейка', 3: 'Кура', 4: 'Свинина', 5: 'Цыпленок'}


In [8]:
class MeatDataset(Dataset):

    def __init__(self, df, tokenizer, label_encoder):
        self.x = df['text']
        self.y = df['mtype']
        self.tokenizer = tokenizer
        self.label_encoder = label_encoder
    
    def __getitem__(self, index):
        item = dict(self.tokenizer(self.x[index], truncation=True, padding='max_length', max_length=512))
        item['label'] = self.label_encoder[self.y[index]]
        #item['label'] = self.tokenizer(self.y[index], truncation=True, padding='max_length', max_length=512)['input_ids']
        return item
        
    def __len__(self):
        return len(self.x)

# Подготовка модели

In [9]:
tokenizer_path = 'cointegrated/rubert-tiny'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

model_path = 'cointegrated/rubert-tiny'
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(label_to_code.keys()))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
train_dataset = MeatDataset(train_df, tokenizer, label_to_code)
test_dataset = MeatDataset(test_df, tokenizer, label_to_code)

In [11]:
SAVE_PATH = "model_data"
training_args = TrainingArguments(
    output_dir=SAVE_PATH,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10, 
    load_best_model_at_end=True,
    eval_strategy = IntervalStrategy.STEPS
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [12]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        
    }

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics
)

trainer.train()
tokenizer.save_pretrained(SAVE_PATH)
model.save_pretrained(SAVE_PATH)    

  0%|          | 0/16440 [00:00<?, ?it/s]c:\Users\admin\PycharmProjects\test_task\myenv\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  3%|▎         | 500/16440 [02:23<1:15:21,  3.53it/s]

{'loss': 0.8368, 'grad_norm': 12.26498031616211, 'learning_rate': 1.9391727493917275e-05, 'epoch': 0.3}


                                                     
  3%|▎         | 500/16440 [02:48<1:15:21,  3.53it/s]

{'eval_loss': 0.2759599983692169, 'eval_accuracy': 0.9239659367396593, 'eval_runtime': 24.7817, 'eval_samples_per_second': 132.679, 'eval_steps_per_second': 16.585, 'epoch': 0.3}


  6%|▌         | 1000/16440 [05:16<1:14:09,  3.47it/s]

...

{'loss': 0.0972, 'grad_norm': 8.495532035827637, 'learning_rate': 1.3309002433090026e-05, 'epoch': 3.35}


                                                    
 33%|███▎      | 5500/16440 [32:32<55:19,  3.30it/s]

{'eval_loss': 0.11734521389007568, 'eval_accuracy': 0.9638077858880778, 'eval_runtime': 26.44, 'eval_samples_per_second': 124.357, 'eval_steps_per_second': 15.545, 'epoch': 3.35}


 33%|███▎      | 5500/16440 [32:33<1:04:45,  2.82it/s]


{'train_runtime': 1953.545, 'train_samples_per_second': 67.314, 'train_steps_per_second': 8.415, 'train_loss': 0.19732488389448685, 'epoch': 3.35}


# Метрики, полученные на тестовых данных

In [14]:
predictions = trainer.predict(test_dataset)

100%|██████████| 411/411 [00:25<00:00, 15.95it/s]


In [15]:
predictions.metrics

{'test_loss': 0.11280577629804611,
 'test_accuracy': 0.9644160583941606,
 'test_runtime': 25.8495,
 'test_samples_per_second': 127.198,
 'test_steps_per_second': 15.9}

# Тестирование на конкретных примерах из тестового задания

In [16]:
def predict(text, device=torch.device('cpu')):
    item = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=512)
    item = {key: value.to(device=device) for key, value in item.items()}
    return code_to_label[model(**item).logits.argmax().item()]

In [17]:
examples = ['Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг.',
            "Мясо премиум Предлагаем котлетное мясо мраморной говядины.",
            "спинка цб",
            'Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)\nСвободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.',
            'Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:',
            'Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.']

for example in examples:
    print(example)
    print(predict(example, device))
    print("_______")

Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг.
Свинина
_______
Мясо премиум Предлагаем котлетное мясо мраморной говядины.
Говядина
_______
спинка цб
Цыпленок
_______
Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)
Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.
Говядина
_______
Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:
Кура
_______
Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.
Говядина
_______
